In [2]:
import bluesky
import bluesky.plan_stubs as bps
from apstools.synApps import SscanRecord
from ophyd import EpicsMotor
from apstools.plans import run_blocking_function
from ophyd.status import Status


In [3]:
IOC = "2idsft:"
scan1 = SscanRecord(f"{IOC}scan1", name="scan1")
scan1.wait_for_connection()

In [6]:
scan1.attr_name

''

In [14]:
m_prefix = "2idbtst:"
m1 = EpicsMotor(f"{m_prefix}m1", name="m1")
m1.wait_for_connection()

In [12]:
def setup_scan1(m1, start, finish, npts, ct=1):
    # yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
    yield from run_blocking_function(scan1.reset)
    yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

    # positioners
    yield from bps.mv(
        scan1.number_points, npts,
        scan1.positioners.p1.start, start,
        scan1.positioners.p1.end, finish,
        scan1.positioners.p1.readback_pv, m1.user_readback.pvname,
        scan1.positioners.p1.setpoint_pv, m1.user_setpoint.pvname,
    )
    # # triggers (push scaler count button at each point)
    # yield from bps.mv(
    #     scan1.triggers.t1.trigger_pv, scaler1.count.pvname,
    # )
    # # detectors
    # yield from bps.mv(
    #     scan1.detectors.d01.input_pv, I0.pvname,
    #     scan1.detectors.d02.input_pv, diode.pvname,
    # )

In [ ]:
def setup_scan1_2D(m1, start, finish, m2, s2, f2, p2, ct=1):
    # yield from bps.mv(scaler1.preset_time, ct)  # counting time/point
    yield from run_blocking_function(scan1.reset)
    yield from bps.sleep(0.2)  # arbitrary wait for EPICS to finish the reset.

    # positioners
    yield from bps.mv(
        scan1.number_points, npts,
        scan1.positioners.p1.start, start,
        scan1.positioners.p1.end, finish,
        scan1.positioners.p1.readback_pv, m1.user_readback.pvname,
        scan1.positioners.p1.setpoint_pv, m1.user_setpoint.pvname,
    )
    # # triggers (push scaler count button at each point)
    # yield from bps.mv(
    #     scan1.triggers.t1.trigger_pv, scaler1.count.pvname,
    # )
    # # detectors
    # yield from bps.mv(
    #     scan1.detectors.d01.input_pv, I0.pvname,
    #     scan1.detectors.d02.input_pv, diode.pvname,
    # )

In [15]:
def run_sscan():
    # Create an instance of the Status object.
    # If it is not marked as finished within 20s, it will raise a timeout exception.
    st = Status(timeout=20)

    def watch_execute_scan(old_value, value, **kwargs):
        # Watch for scan1.EXSC to change from 1 to 0 (when the scan ends).
        if old_value == 1 and value == 0:
            # mark as finished (successfully).
            st.set_finished()
            # Remove the subscription.
            scan1.execute_scan.clear_sub(watch_execute_scan)

    yield from bps.mv(scan1.execute_scan, 1)
    scan1.execute_scan.subscribe(watch_execute_scan)
    yield from run_blocking_function(st.wait)

In [2]:
# RE = bluesky.RunEngine()
# RE = bluesky.RunEngine()
a = RE(setup_scan1(m1, -0.5, 0.5, 100))

ModuleNotFoundError: No module named 'bluesky.run_engine.RunEngine'; 'bluesky.run_engine' is not a package

In [34]:
# Run scan
j = RE(run_sscan())

In [31]:
# This command interacts with pv via ophyd directly, without bluesky RE
scan1.execute_scan.value = 1

In [14]:
scan1.number_points.pvname

'2idsft:scan1.NPTS'

In [7]:
from ophyd import Signal

In [39]:
scan_npts = Signal(name='2idsft:scan1.NPTS')

In [27]:
print(scan1.number_points.value)
scan_npts.put = 100
print(scan1.number_points.value)

1
1


In [19]:
type(scan_npts)

ophyd.signal.Signal

In [28]:
scan1.number_points.value = 100

In [40]:
scan_npts.get

<bound method Signal.get of Signal(name='2idsft:scan1.NPTS', value=0.0, timestamp=1713302585.1919556)>

CA.Client.Exception...............................................
    Context: "2iddxspress3.xray.aps.anl.gov:44487"
    Source File: ../cac.cpp line 1238
    Current Time: Tue Apr 16 2024 16:43:03.154788457
..................................................................


In [48]:
scan1.positioners.p1.value = '2idsft:m2'

In [49]:
import numpy as np

In [79]:
t =np.linspace(-0.5, 0.5, 2000)

In [70]:
t

array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
       -0.23684211, -0.18421053, -0.13157895, -0.07894737, -0.02631579,
        0.02631579,  0.07894737,  0.13157895,  0.18421053,  0.23684211,
        0.28947368,  0.34210526,  0.39473684,  0.44736842,  0.5       ])

In [68]:
scan1.positioners

Positioners(prefix='2idsft:scan1', name='scan1_positioners', parent='scan1', read_attrs=['p1', 'p1.readback_value', 'p1.array', 'p1.setpoint_value', 'p2', 'p2.readback_value', 'p2.array', 'p2.setpoint_value', 'p3', 'p3.readback_value', 'p3.array', 'p3.setpoint_value', 'p4', 'p4.readback_value', 'p4.array', 'p4.setpoint_value'], configuration_attrs=['p1', 'p1.readback_pv', 'p1.setpoint_pv', 'p1.start', 'p1.center', 'p1.end', 'p1.step_size', 'p1.width', 'p1.abs_rel', 'p1.mode', 'p1.units', 'p2', 'p2.readback_pv', 'p2.setpoint_pv', 'p2.start', 'p2.center', 'p2.end', 'p2.step_size', 'p2.width', 'p2.abs_rel', 'p2.mode', 'p2.units', 'p3', 'p3.readback_pv', 'p3.setpoint_pv', 'p3.start', 'p3.center', 'p3.end', 'p3.step_size', 'p3.width', 'p3.abs_rel', 'p3.mode', 'p3.units', 'p4', 'p4.readback_pv', 'p4.setpoint_pv', 'p4.start', 'p4.center', 'p4.end', 'p4.step_size', 'p4.width', 'p4.abs_rel', 'p4.mode', 'p4.units'])

In [87]:
dir(scan1)

['OphydAttrList',
 'SUB_ACQ_DONE',
 '_Device__set_kinds_according_to_list',
 '_OphydObject__any_instantiated',
 '_OphydObject__instantiation_callbacks',
 '_OphydObject__register_instance',
 '__annotations__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getnewargs_ex__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_args_cache',
 '_attr_name',
 '_callbacks',
 '_cb_count',
 '_cid_to_event_mapping',
 '_component_kinds',
 '_default_configuration_attrs',
 '_default_read_attrs',
 '_default_sub',
 '_destroyed',
 '_device_tuple',
 '_done_acquiring',
 '_get_components_of_kind',
 '_get_kind',
 '_initialize_device',
 '_instantiate_component',
 '_kind',
 '_mark_as_instan

In [75]:
from epics import caget, caput

In [100]:
a = caget('2idsft:scan1.P1PA')
a

array([-0.5       , -0.44736842, -0.39473684, -0.34210526, -0.28947368,
       -0.23684211, -0.18421053, -0.13157895, -0.07894737, -0.02631579,
        0.02631579,  0.07894737,  0.13157895,  0.18421053,  0.23684211,
        0.28947368,  0.34210526,  0.39473684,  0.44736842,  0.5       ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [97]:
t =np.linspace(-0.5, 0.5, 20)

In [98]:
caput('2idsft:scan1.P1PA', t)

1

In [99]:
caput('2idsft:scan1.P2PA', t)

1

In [90]:
scan1.positioner_delay

EpicsSignal(read_pv='2idsft:scan1.PDLY', name='scan1_positioner_delay', parent='scan1', value=0.10000000149011612, timestamp=631152000.0, tolerance=0.001, auto_monitor=True, string=False, write_pv='2idsft:scan1.PDLY', limits=False, put_complete=False)

In [102]:
t.shape

(20,)

In [103]:
xx, yy = np.meshgrid(t,t)
xx.

(20, 20)

In [104]:
xx.flatten().shape

(400,)